## import library

In [1]:
import os
import numpy as np
import tensorflow as tf
from data_loader import *
from data_utils import *
from model import *
from const import *

c:\users\shimd\anaconda3\envs\mondeique\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\shimd\anaconda3\envs\mondeique\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\shimd\anaconda3\envs\mondeique\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\shimd\anaconda3\envs\mondeique\lib\site-packages\ten

## prepare data

In [2]:
train_data, test_data = getBagImage()

Reading csv................
Load bag image...................
Number of bag train data:  1993
Number of bag test data:  478
---------------------------------------------------------------


In [3]:
def one_hot(index, num_classes):
    tmp = np.zeros(num_classes, dtype=np.float32)
    tmp[index] = 1.0
    return tmp

# main evaluation code

## input tensor + model network

In [4]:
sess = tf.InteractiveSession()
x, y, mask = input_tensor()

y_color_conv, y_shape_conv, y_opening_conv, y_strap_conv, y_pattern_conv, y_carrying_conv, y_charm_conv, y_decoration_conv, is_training, keep_prob = multi_label_net(x)

W0819 10:30:38.076784 11096 deprecation_wrapper.py:119] From C:\Users\shimd\Desktop\multi-label-classification\model.py:7: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0819 10:30:38.080797 11096 deprecation_wrapper.py:119] From C:\Users\shimd\Desktop\multi-label-classification\ops.py:253: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0819 10:30:38.081771 11096 deprecation_wrapper.py:119] From C:\Users\shimd\Desktop\multi-label-classification\ops.py:141: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0819 10:30:38.162577 11096 deprecation_wrapper.py:119] From C:\Users\shimd\Desktop\multi-label-classification\ops.py:205: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0819 10:30:38.408830 11096 deprecation_wrapper.py:119] From C:\Users\shimd\Desktop\multi-label-classification\ops.py:181: The name tf.nn.xw_plus_b is deprecated. Pleas

## loss + mask

In [5]:
color_loss, shape_loss, opening_loss, strap_loss, pattern_loss, carrying_loss, charm_loss, decoration_loss, l2_loss, total_loss = selective_loss(y_color_conv, y_shape_conv, y_opening_conv, y_strap_conv, y_pattern_conv, y_carrying_conv, y_charm_conv, y_decoration_conv,y, mask)

color_mask = tf.get_collection('color_mask')[0]
shape_mask = tf.get_collection('shape_mask')[0]
opening_mask = tf.get_collection('opening_mask')[0]
strap_mask = tf.get_collection('strap_mask')[0]
pattern_mask = tf.get_collection('pattern_mask')[0]
carrying_mask = tf.get_collection('carrying_mask')[0]
charm_mask = tf.get_collection('charm_mask')[0]
decoration_mask = tf.get_collection('decoration_mask')[0]

y_color = tf.get_collection('y_color')[0]
y_shape = tf.get_collection('y_shape')[0]
y_opening = tf.get_collection('y_opening')[0]
y_strap = tf.get_collection('y_strap')[0]
y_pattern = tf.get_collection('y_pattern')[0]
y_carrying = tf.get_collection('y_carrying')[0]
y_charm = tf.get_collection('y_charm')[0]
y_decoration = tf.get_collection('y_decoration')[0]

color_correct_prediction = tf.equal(tf.argmax(y_color_conv, 1), tf.argmax(y_color, 1))
shape_correct_prediction = tf.equal(tf.argmax(y_shape_conv, 1), tf.argmax(y_shape, 1))
opening_correct_prediction = tf.equal(tf.argmax(y_opening_conv, 1), tf.argmax(y_opening, 1))
strap_correct_prediction = tf.equal(tf.argmax(y_strap_conv, 1), tf.argmax(y_strap, 1))
pattern_correct_prediction = tf.equal(tf.argmax(y_pattern_conv, 1), tf.argmax(y_pattern, 1))
carrying_correct_prediction = tf.equal(tf.argmax(y_carrying_conv, 1), tf.argmax(y_carrying, 1))
charm_correct_prediction = tf.equal(tf.argmax(y_charm_conv, 1), tf.argmax(y_charm, 1))
decoration_correct_prediction = tf.equal(tf.argmax(y_decoration_conv, 1), tf.argmax(y_decoration, 1))

color_true_pred = tf.reduce_sum(tf.cast(color_correct_prediction, dtype=tf.float32) * color_mask)
shape_true_pred = tf.reduce_sum(tf.cast(shape_correct_prediction, dtype=tf.float32) * shape_mask)
opening_true_pred = tf.reduce_sum(tf.cast(opening_correct_prediction, dtype=tf.float32) * opening_mask)
strap_true_pred = tf.reduce_sum(tf.cast(strap_correct_prediction, dtype=tf.float32) * strap_mask)
pattern_true_pred = tf.reduce_sum(tf.cast(pattern_correct_prediction, dtype=tf.float32) * pattern_mask)
carrying_true_pred = tf.reduce_sum(tf.cast(carrying_correct_prediction, dtype=tf.float32) * carrying_mask)
charm_true_pred = tf.reduce_sum(tf.cast(charm_correct_prediction, dtype=tf.float32) * charm_mask)
decoration_true_pred = tf.reduce_sum(tf.cast(decoration_correct_prediction, dtype=tf.float32) * decoration_mask)

real_test_data = []

# Mask : color -> 0 , shape -> 1, opening -> 2, strap -> 3, pattern -> 4, carrying -> 5, charm -> 6, decoration -> 7
for i in range(len(test_data)):
    img = (test_data[i][0] - 128) / 255.0
    label = test_data[i][1]
    real_test_data.append((img, one_hot(label[i][0], 16), 0.0))
for i in range(len(test_data)):
    img = (test_data[i][0] - 128) / 255.0
    label = test_data[i][1]
    real_test_data.append((img, one_hot(label[i][1], 16), 1.0))
for i in range(len(test_data)):
    img = (test_data[i][0] - 128) / 255.0
    label = test_data[i][1]
    real_test_data.append((img, one_hot(label[i][2], 16), 2.0))
for i in range(len(test_data)):
    img = (test_data[i][0] - 128) / 255.0
    label = test_data[i][1]
    real_test_data.append((img, one_hot(label[i][3], 16), 3.0))
for i in range(len(test_data)):
    img = (test_data[i][0] - 128) / 255.0
    label = test_data[i][1]
    real_test_data.append((img, one_hot(label[i][4], 16), 4.0))
for i in range(len(test_data)):
    img = (test_data[i][0] - 128) / 255.0
    label = test_data[i][1]
    real_test_data.append((img, one_hot(label[i][5], 16), 5.0))
for i in range(len(test_data)):
    img = (test_data[i][0] - 128) / 255.0
    label = test_data[i][1]
    real_test_data.append((img, one_hot(label[i][6], 16), 6.0))
for i in range(len(test_data)):
    img = (test_data[i][0] - 128) / 255.0
    label = test_data[i][1]
    real_test_data.append((img, one_hot(label[i][7], 16), 7.0))

np.random.shuffle(real_test_data)

print('Restore model')
saver = tf.train.Saver()
saver.restore(sess, './save/current2/model_v3.ckpt')
print('OK')

W0819 10:30:38.862823 11096 deprecation_wrapper.py:119] From C:\Users\shimd\Desktop\multi-label-classification\model.py:94: The name tf.add_to_collection is deprecated. Please use tf.compat.v1.add_to_collection instead.

W0819 10:30:38.875751 11096 deprecation_wrapper.py:119] From C:\Users\shimd\Desktop\multi-label-classification\model.py:122: The name tf.log is deprecated. Please use tf.math.log instead.

W0819 10:30:39.285268 11096 deprecation.py:323] From c:\users\shimd\anaconda3\envs\mondeique\lib\site-packages\tensorflow\python\training\saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Restore model
OK


In [8]:
train_img = []
train_label = []
train_mask = []

for i in range(len(real_test_data)):
    train_img.append(real_test_data[i][0])
    train_label.append(real_test_data[i][1])
    train_mask.append(real_test_data[i][2])

number_batch = len(real_test_data) // BATCH_SIZE

color_nb_true_pred = 0
shape_nb_true_pred = 0
opening_nb_true_pred = 0
strap_nb_true_pred = 0
pattern_nb_true_pred = 0
carrying_nb_true_pred = 0
charm_nb_true_pred = 0
decoration_nb_true_pred = 0

color_nb_train = 0
shape_nb_train = 0
opening_nb_train = 0
strap_nb_train = 0
pattern_nb_train = 0
carrying_nb_train = 0
charm_nb_train = 0
decoration_nb_train = 0

for batch in range(number_batch):
    # print('Training on batch {0}/{1}'.format(str(batch + 1), str(number_batch)))
    top = batch * BATCH_SIZE
    bot = min((batch + 1) * BATCH_SIZE, len(real_test_data))
    batch_img = np.asarray(train_img[top:bot])
    batch_label = np.asarray(train_label[top:bot])
    batch_mask = np.asarray(train_mask[top:bot])
    
    batch_img = augmentation(batch_img, 64)

    for i in range(BATCH_SIZE):
        if batch_mask[i] == 0.0:
            color_nb_train += 1
        else:
            if batch_mask[i] == 1.0:
                shape_nb_train += 1
            else:
                if batch_mask[i] == 2.0:
                    opening_nb_train += 1
                else:
                    if batch_mask[i] == 3.0:
                        strap_nb_train += 1
                    else:
                        if batch_mask[i] == 4.0:
                            pattern_nb_train += 1
                        else:
                            if batch_mask[i] == 5.0:
                                carrying_nb_train += 1
                            else:
                                if batch_mask[i] == 6.0:
                                    charm_nb_train += 1
                                else:
                                    decoration_nb_train +=1

      

    color_nb_true_pred += sess.run(color_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                               is_training: False,keep_prob: 1})

    shape_nb_true_pred += sess.run(shape_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                               is_training: False, keep_prob: 1})

    opening_nb_true_pred += sess.run(opening_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                   is_training: False, keep_prob: 1})

    strap_nb_true_pred += sess.run(strap_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                               is_training: False, keep_prob: 1})

    pattern_nb_true_pred += sess.run(pattern_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                   is_training: False, keep_prob: 1})

    carrying_nb_true_pred += sess.run(carrying_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                     is_training: False, keep_prob: 1})

    charm_nb_true_pred += sess.run(charm_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                     is_training: False, keep_prob:1})

    decoration_nb_true_pred += sess.run(decoration_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                     is_training: False, keep_prob: 1})

    sess.run(update_op,feed_dict={x: batch_img, y_: batch_label, mask: batch_mask, is_training: False, keep_prob: 1})



color_train_accuracy = color_nb_true_pred * 1.0 / color_nb_train
shape_train_accuracy = shape_nb_true_pred * 1.0 / shape_nb_train
opening_train_accuracy = opening_nb_true_pred * 1.0 / opening_nb_train
strap_train_accuracy = strap_nb_true_pred * 1.0 / strap_nb_train
pattern_train_accuracy = pattern_nb_true_pred * 1.0 / pattern_nb_train
carrying_train_accuracy = carrying_nb_true_pred * 1.0 / carrying_nb_train
charm_train_accuracy = charm_nb_true_pred * 1.0 / charm_nb_train
decoration_train_accuracy = decoration_nb_true_pred * 1.0 / decoration_nb_train

print('\n')
print('color task train accuracy: ' + str(color_train_accuracy * 100))
print('shape task train accuracy: ' + str(shape_train_accuracy * 100))
print('opening task train accuracy: ' + str(opening_train_accuracy * 100))
print('strap task train accuracy: ' + str(strap_train_accuracy * 100))
print('pattern task train accuracy: ' + str(pattern_train_accuracy * 100))
print('carrying task train accuracy: ' + str(carrying_train_accuracy * 100))
print('charm task train accuracy: ' + str(charm_train_accuracy * 100))
print('decoration task train accuracy: ' + str(decoration_train_accuracy * 100))

NameError: name 'update_op' is not defined